# Training and Evaluating Models
In this notebook, we will learn how to train a defined model and evalute its performance.
* Objectives: Learning built-in training (adam, ada), evaluating (accuracy) functions
* Prerequisites: Knet Neural Network Architecture and Layers notebook


Importing Knet:

In [ ]:
#using Pkg
#Pkg.add("Knet")
using Knet
using MLDatasets
#You may see an error if your device does not support CUDA or your CUDA driver is not CUDA 10.1 or higher but you will be
#able to use all the functionalities, except GPU operations, in spite of this error

Let's remember our layer and model definitions from the last tutorial:

In [ ]:
struct dense; w; b; f; end
(d::dense)(x) = d.f.(d.w * mat(x) .+ d.b)
dense(i::Int,o::Int,f=relu) = dense(param(o,i), param0(o), f);

struct Conv; w; b; f; end
(c::Conv)(x) = c.f.(pool(conv4(c.w, x) .+ c.b))
Conv(w1,w2,cx,cy,f=relu) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f);

struct Chain; layers; Chain(args...)= new(args);end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain)(x,y) = nll(c(x),y)

LeNet = Chain(Conv(5,5,1,20), Conv(5,5,20,50), dense(800,500), dense(500,10,identity))

The data we need to train the model will be imported from an open source Julia library [MLDatasets](https://github.com/JuliaML/MLDatasets.jl). Details of importing data and using built-in Knet utilities for preprocessing will be explained in depth later.

In [ ]:
# Load MNIST data
xtrn,ytrn = MNIST.traindata(Float32); ytrn[ytrn.==0] .= 10
xtst,ytst = MNIST.testdata(Float32);  ytst[ytst.==0] .= 10
dtrn = minibatch(xtrn, ytrn, 100; xsize=(size(xtrn,1),size(xtrn,2),1,:))
dtst = minibatch(xtst, ytst, 100; xsize=(size(xtst,1),size(xtst,2),1,:));

In Knet, we pass the model and data to optimizer functions instead of conventional "model.train" way.
Built-in optimization functions:
* adam
* adadelta
* momentum
* rmsprop
* adagrad
* nesterov

An in-depth explanation of the optimization algorithms:

In [ ]:
@doc adam

### Training

In [ ]:
adam(LeNet, ncycle(dtrn,10))

The ncycle function is built in [IterTools](https://juliacollections.github.io/IterTools.jl/latest/) library that takes the data as the first parameter and the number of epochs as the second parameter. 

In [ ]:
@doc nycle

When trained (it will take 30 seconds to 10 minutes depending on the CPU/GPU power), you will see that the function does not visualize the progress. For that, Knet has a function named progress!

In [ ]:
@doc progress!

In [ ]:
progress!(adam(LeNet, ncycle(dtrn,10)))

Knet also has the function "train!" which is deprecated but still fully functional:

function train!(model, data; loss=nll, optimizer=Adam(), callback=epochs(data,1), o...)

In [ ]:
progress!(train!(model, dtrn))